# 二、各省市知识产权官方微博数据的抓取

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
import sys

In [2]:
import os
import requests
from selenium import webdriver

# 引入 chromedriver.exe
chromedriver = r"C:\Users\谢海婷\Desktop\python实验\chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
browser = webdriver.Chrome(chromedriver)

# 设置浏览器需要打开的 url
url = "https://weibo.com/"
browser.get(url)

# 隐式等待
browser.implicitly_wait(20)

# 获取组件并传值
browser.find_element_by_id("loginname").send_keys("******")   #*****填入自己的用户名
browser.find_element_by_name("password").send_keys("*******")     #*****填入自己的密码
browser.find_element_by_class_name("btn_32px").click()

browser.implicitly_wait(20)

#出现输入验证码操作
try:
    if '"text=请输入验证码"' in browser.page_source:
        img_ele=browser.find_element_by_xpath('//img[@node-type="verifycode_image"]')
        image_url = img_ele.get_attribute('src')
        response=requests.get(image_url)
        with open('verify_image.jpg', 'wb') as f:
            f.write(response.content)
        vares_img=input('请输入：\n')
        browser.find_element_by_name('verifycode').send_keys(vares_img)
        browser.find_element_by_class_name('W_btn_a ').click()
except:
    pass

In [3]:
#检验是否登录成功，获取cookies
cookie = []
if "微博-随时随地发现新鲜事" in browser.title:
    for elem in browser.get_cookies():
        s3="%s=%s"%(elem["name"],elem["value"])
        cookie.append(s3)
    cookieStr = ";".join(cookie)
    print('成功登录微博')
    browser.implicitly_wait(10)
else:
    print("登录 Cookie Failed: !" )

成功登录微博


In [4]:
cookieStr

'TC-Page-G0=153ff31dae1cf71cc65e7e399bfce283|1575939805|1575939805;wvr=6;un=17316861446;SUHB=0WGcY68suVx1I6;SUB=_2A25w6p6EDeRhGeBL6VsY9S7OyDmIHXVTgfdMrDV8PUNbmtANLWvDkW9NRzxvBgHizk7G9eNglOF-MIXCkSQWoG9o;WBStorage=42212210b087ca50|undefined;SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFNS0d4ZumjBx2zjiLULykH5JpX5K2hUgL.Foqfeo.4SK5Ee0-2dJLoIpqLxKqLBo-LBoU2xsLyqNHX9K.0ShBN;TC-V5-G0=595b7637c272b28fccec3e9d529f251a;SINAGLOBAL=1164422121286.0173.1575939791726;webim_unReadCount=%7B%22time%22%3A1575939806567%2C%22dm_pub_total%22%3A0%2C%22chat_group_client%22%3A0%2C%22allcountNum%22%3A31%2C%22msgbox%22%3A0%7D;Apache=1164422121286.0173.1575939791726;SCF=AhhdwBDJ3Wq0XjT1uPc-7_k4pSQ8jL4jiqjr2wRsqMWjDo79a7YwEo2A9QY__ktcnxvK0Ix8MT-YIqjUj6Os4qg.;_s_tentry=passport.weibo.com;wb_view_log=1536*8641.25;ULV=1575939791743:1:1:1:1164422121286.0173.1575939791726:;ALF=1607475793;cross_origin_proto=SSL;SSOLoginState=1575939795;login_sid_t=0c6476ffde1e7c462326fca5e9e71c8e;wb_view_log_6529950235=1536*8641.25;Ugrow-G0=

In [5]:
import requests
import time as t 
#对每页的所有微博内容进行获取请求得到网页的文本
def urlRequest(pagebar,page,pre_page):
    url = 'https://weibo.com/p/aj/v6/mblog/mbloglist'
    headers ={
    'cookie':cookieStr,
    'host': 'weibo.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
    }
    global tt
    params = {
    'ajwvr': '6',
    'domain': '100106',
    'profile_ftype': '1',
    'is_all': '1',
    'pl_name': 'Pl_Official_MyProfileFeed__24',
    'feed_type': '0',
    'domain_op': '100106',
    ##
    'pagebar':pagebar,
    'id': '1001062611704875',
    'script_uri': '/u/2611704875',
    'page': page,
    'pre_page': pre_page,
    '__rnd':'{:.3f}'.format(t.time()).replace('.',''),
    }
    res=requests.get(url,headers=headers,params=params)
    #res.status_code
    return res

In [6]:
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
#在请求回复的微博内容中提取所需要的
def run(res):
    json_res = res.json()
    html = json_res.get('data')
    soup = BeautifulSoup(html,"lxml")
    n = 0
    for w in soup.find_all("div",'WB_cardwrap WB_feed_type S_bg2 WB_feed_like'):
        web={}
        try:
            str = w.find_all("div",'WB_text W_f14')[0].contents[2].replace('\n','').replace(' ','') if n==0 else w.find_all("div",'WB_text W_f14')[0].contents[0].replace('\n','').replace(' ','')

        except:
            str = w.find_all("div",'WB_text W_f14')[0].contents[0].replace('\n','').replace(' ','')
        #str
        if '【' and '】' in str:
            #通过检验获取文本中是否存在‘展开全文’来判断微博内容是否完整并通过mid值来重新请求来获取全文内容
            if re.search(r'展开全文','{}'.format(w.find_all("div",'WB_text W_f14')[0])):    
                url_='https://weibo.com/p/aj/mblog/getlongtext'
                headers_={
                    'cookie':cookieStr,
                    'Referer': 'https://weibo.com/u/2611704875?profile_ftype=1&is_all=1',
                    'Host': 'weibo.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
                }
                params1={
                    'ajwvr': '6',
                    'mid': w.get('mid'),
                    'is_settop': '',
                    'is_sethot': '',
                    'is_setfanstop': '',
                    'is_setyoudao': '',
                    '__rnd': '{:.3f}'.format(t.time()).replace('.',''),
                }
                r = requests.get(url_,params=params1,headers=headers_)
                json_r = r.json()
                html_r = json_r['data']['html']
                content = BeautifulSoup(html_r,"lxml").a.previous_sibling
            else:
                content = str
        else:
            n = n+1
            continue
        #content
        try:
            time = w.find_all("div",'WB_from S_txt2')[0].find_all('a')[0].get('title')
            source = w.find_all("div",'WB_from S_txt2')[0].find_all('a')[1].string
            forward = '0' if w.find_all("div",'WB_handle')[0].find_all('em')[3].string == '转发' else w.find_all("div",'WB_handle')[0].find_all('em')[3].string #forward_btn_text
            comment = '0'if w.find_all("div",'WB_handle')[0].find_all('em')[6].string == '评论' else w.find_all("div",'WB_handle')[0].find_all('em')[6].string#comment_btn_text
            like = '0' if w.find_all("div",'WB_handle')[0].find_all('em')[9].string == '赞' else w.find_all("div",'WB_handle')[0].find_all('em')[9].string#like_status
        except:
            continue
        web['微博内容']=content
        web['时间']=time
        web['来源']=source
        web['转发数']=forward
        web['评论数']=comment
        web['点赞数']=like
        data.append(web)
        n = n+1

In [7]:
#爬取10页内容
data=[]
for i in range(1,11):    
    res = urlRequest('0','{}'.format(i),'{}'.format(i-1))
    res1 = urlRequest('0','{}'.format(i),'{}'.format(i))
    res2 = urlRequest('1','{}'.format(i),'{}'.format(i))
    run(res)
    run(res1)
    run(res2)
    print("第{}页爬取成功！".format(i))
df = pd.DataFrame(data)
df 

第1页爬取成功！
第2页爬取成功！
第3页爬取成功！
第4页爬取成功！
第5页爬取成功！
第6页爬取成功！
第7页爬取成功！
第8页爬取成功！
第9页爬取成功！
第10页爬取成功！


,微博内容,时间,来源,转发数,评论数,点赞数
0,【与你有关！《北京市知识产权资助金管理办法（试行）》正式出台，将有效帮助创新主体降成本】为鼓...,2019-12-09 16:42,政务直通车,4,0,0
1,【北京市知识产权局传达学习市委十二届十次全会精神】近日，北京市知识产权局召开党组（扩大）会，...,2019-12-09 16:41,政务直通车,0,0,0
2,【中国（北京）知识产权保护中心、北京专利代办处联合开展专利周窗口开放日活动】11月30日-1...,2019-12-09 16:40,政务直通车,0,0,1
3,【中关村知识产权帮扶系列活动——视联动力信息技术股份有限公司专场活动成功举办】12月5日上午...,2019-12-09 15:18,政务直通车,2,0,0
4,【中关村知识产权帮扶系列活动——北京旷视科技有限公司专场活动成功举办】12月5日上午，中关村...,2019-12-09 15:16,政务直通车,2,0,1
5,【中关村知识产权帮扶系列活动——大唐移动通信设备有限公司专场活动成功举办】12月5日下午，中...,2019-12-09 15:16,政务直通车,2,0,0
6,【中关村科学城成立开源芯片创新中心】瞄准芯片设计的“金字塔尖”和源头创新，12月7日下午，中...,2019-12-09 15:09,政务直通车,4,0,0
7,【北京未来科学城东区将建“能源谷”】近日，由北京市人民政府和国务院发展研究中心主办的“201...,2019-12-09 15:08,政务直通车,0,0,0
8,【313家企业新晋2019年度北京市知识产权试点示范单位行列，欢迎来点赞！】根据《北京市知识...,2019-12-06 14:49,政务直通车,3,0,2
9,【北京市地方标准《企业知识产权管理规范（征求意见稿）》开始征求意见啦！】北京市地方标准《企业...,2019-12-06 14:46,政务直通车,1,0,0


In [8]:
# 关闭浏览器
browser.quit()

In [9]:
#将数据存入excel
df.to_excel('Weibo.xls',index = False)